In [22]:
!jupyter labextension list

JupyterLab v3.2.4
/home/fterroso/.local/share/jupyter/labextensions
        @jupyter-widgets/jupyterlab-manager v3.1.0 enabled OK (python, jupyterlab_widgets)



In [23]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [24]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/bin/python3
3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)


In [25]:
!pip install pandas numpy tqdm scikit-learn

!pip install torch==1.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install ipywidgets

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [26]:
import os 

import pandas as pd
import numpy as np 
import json
from tqdm.notebook import tqdm
import pickle

from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data, Dataset
import torch

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU, GCLSTM, DCRNN, A3TGCN

from sklearn.metrics import accuracy_score, classification_report

#for local environment
data_path= os.path.join('data', 'mobility')

import warnings
warnings.filterwarnings('ignore')

In [27]:
import torch
torch.version.cuda

'11.3'

In [28]:
torch.cuda.empty_cache()

In [29]:
from importlib import reload
import tourist_mobility3
reload(tourist_mobility3)
from tourist_mobility3 import TouristMobDynamicDatasetLoader

In [30]:
with open(os.path.join('data', 'graph_datasets', 'node_to_codes_all_data_dynamic.json')) as f:
     nodes_to_code= json.load(f)

In [31]:
nodes_to_code

{'0': '01001_AM',
 '1': '01002',
 '2': '01010_AM',
 '3': '01031_AM',
 '4': '01036',
 '5': '01043_AM',
 '6': '01047_AM',
 '7': '01051',
 '8': '01058_AM',
 '9': '01059',
 '10': '01063_AM',
 '11': '01901_AM',
 '12': '02003',
 '13': '02008_AM',
 '14': '02009',
 '15': '02010_AM',
 '16': '02012_AM',
 '17': '02021_AM',
 '18': '02024_AM',
 '19': '02025',
 '20': '02029_AM',
 '21': '02030_AM',
 '22': '02033_AM',
 '23': '02037',
 '24': '02045_AM',
 '25': '02053_AM',
 '26': '02063_AM',
 '27': '02067_AM',
 '28': '02069_AM',
 '29': '02073_AM',
 '30': '02074',
 '31': '02079_AM',
 '32': '02081',
 '33': '02086_AM',
 '34': '03002_AM',
 '35': '03005_AM',
 '36': '03009',
 '37': '03011',
 '38': '03014',
 '39': '03015',
 '40': '03018',
 '41': '03019',
 '42': '03021',
 '43': '03024',
 '44': '03028_AM',
 '45': '03031',
 '46': '03041',
 '47': '03043_AM',
 '48': '03046_AM',
 '49': '03047',
 '50': '03048',
 '51': '03049',
 '52': '03050',
 '53': '03053',
 '54': '03055',
 '55': '03056',
 '56': '03058_AM',
 '57': '

## Compose GNN

In [32]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(RecurrentGCN, self).__init__()
        self.recurrent1 = A3TGCN(in_channels=node_features, out_channels=128, periods=periods) # node_features=4, periods=7
        self.linear1 = torch.nn.Linear(128, 128)
        self.dropout = torch.nn.Dropout(0.25)    
        self.linear2 = torch.nn.Linear(128, 128)

        self.linear3 = torch.nn.Linear(128, 8)

    def forward(self, x, edge_index, edge_weight, h):
        #print(x.shape, edge_index.shape, edge_weight.shape)
        h_0= self.recurrent1(x, edge_index, edge_weight, h)
        h = F.relu(h_0)
        
        h = self.linear1(h)
        h = F.relu(h)

        h = self.linear2(h)
        h = F.relu(h)
        #h2= self.linear2(torch.Tensor(x[1]))
        #h3 = torch.cat((h1, h2), dim=1)
        h= self.dropout(h)
        h = self.linear3(h)
        #print(self.linear3.weight.grad)

        return h, h_0

#### Train GNN

Support functions 

In [33]:
def sample_negative_edges(positive_edges, nodes_to_code, n_samples):
    
    edges_= np.vstack((positive_edges[0].cpu(),positive_edges[1].cpu())).T
    
    origins = []
    dest = []
    #Here we generate a negative link for each node in the snapshot
    for i in range(n_samples):
        
        o_node= np.random.randint(len(nodes_to_code.keys()))
        d_node= np.random.randint(len(nodes_to_code.keys()))

        edge_is_new= (len(edges_[(edges_[:,0]==o_node) & (edges_[:,1]==d_node)])==0)
        
        #If the (o_node, d_node) tuple already exists, try again...
        while not edge_is_new:
            d_node= np.random.randint(len(nodes_to_code.keys()))
        
            edge_is_new= (len(edges_[(edges_[:,0]==o_node) & (edges_[:,1]==d_node)])==0)

        origins.append(o_node)
        dest.append(d_node)
        
    edge_index_negs = torch.row_stack([torch.LongTensor(origins), torch.LongTensor(dest)])
    return edge_index_negs

def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    onehot = np.eye(unique.shape[0])[inverse]
    return torch.Tensor(onehot)

def get_labels_for_edges(edges_subset, all_edges, all_labels):
    
    edges_subset_array= edges_subset.numpy()
    edges_subset_array= np.stack(edges_subset_array).T
    
    a= all_edges.numpy()
    a=np.stack(a).T

    locations = [np.where((a[:,0]==o) & (a[:,1]==d)) for (o,d) in edges_subset_array]
    #print(all_labels)
    location_lst= [l[0][0] for l in locations]
    
    return all_labels[location_lst]

def predict_scores(edge_index, embs):
    scores = embs[edge_index[0,:], :] * embs[edge_index[1,:], :] # taking dot product for each playlist/song pair
    scores = scores.sum(dim=1)
    scores = torch.sigmoid(scores)
    return scores

In [34]:
def train(train_dataset, T, device, GNN_type='regular', max_snapshots=None):
    
    model = RecurrentGCN(node_features = 5, periods= 7)
     
    opt = torch.optim.Adam(model.parameters(), lr=0.01)

    model = model.to(device)
    
    neg_edges_rate=1
    n_epochs=40

    softmax = torch.nn.Softmax(dim=1)
    batch_size= 10

    # Early stopping
    the_last_loss = 100
    patience = 6
    trigger_times = 0

    model.train()

    snapshots_dict= dict((i,j) for i,j in enumerate(train_dataset))
    time_steps= np.sort(list(snapshots_dict.keys()))

    T=1 #Time horizon
    loss_fn = torch.nn.BCELoss()
    #torch.autograd.set_detect_anomaly(True)
    
    range_low_limit=0
    n_timestamps = len(time_steps)
    if max_snapshots:
        range_low_limit=max_snapshots
        n_timestamps = max_snapshots
        
    print(f"The number of timesteps for training is {n_timestamps}")

    #for epoch in tqdm(range(n_epochs), desc='Epochs...', leave=False):
    for epoch in range(n_epochs):
        #torch.cuda.empty_cache()
        cost =0
        node_embeddings = None
        h= None
        counter= 1
        batch_loss = 0
        last_t = 0
        #the_current_loss = 0
        #for t in tqdm(time_steps[:-T], desc='Time steps...', leave=False):
        for t in time_steps[-(range_low_limit+T):-T]:
            
            train_snapshot = snapshots_dict[t].to(device)
            target_snapshot = snapshots_dict[t+T]
            
            #Generate embeddings from the present graph
            node_embeddings, h= model(train_snapshot.x, 
                                      train_snapshot.edge_index, 
                                      train_snapshot.edge_attr,          
                                      h)

            transform = RandomLinkSplit(is_undirected=False, add_negative_train_samples=False, neg_sampling_ratio=0,
                                        num_val=0.15, num_test=0)

            train_split, val_split, _ = transform(target_snapshot)

            #Extract labels for positive edges and generate "non-exist" labels for the negative labels
            pos_edges = train_split.edge_index

            #Calculate train loss
            #Genereate negative labels by considering the target graph at t+T 
            neg_edges_to_sample = int(pos_edges.shape[1]*neg_edges_rate)
            neg_edges = sample_negative_edges(pos_edges, nodes_to_code, neg_edges_to_sample)

            pos_scores= predict_scores(pos_edges, node_embeddings)
            neg_scores= predict_scores(neg_edges, node_embeddings)

            all_scores = torch.cat([pos_scores, neg_scores], dim=0)
            all_labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])], dim=0)

            #print(node_embeddings, all_scores, all_labels)
            loss = loss_fn(all_scores.to(device), all_labels.to(device))
            cost = cost + loss
            """
            #Calculate validation loss for early stopping
            val_loss=0
            with torch.no_grad():
                neg_edges_to_sample = int(val_split.edge_index.shape[1]*neg_edges_rate) #100% of negative edges
                neg_edges = sample_negative_edges(val_split.edge_index, nodes_to_code, neg_edges_to_sample)

                pos_scores= predict_scores(val_split.edge_index, node_embeddings)
                neg_scores= predict_scores(neg_edges, node_embeddings)

                all_scores = torch.cat([pos_scores, neg_scores], dim=0)
                all_labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])], dim=0)

                val_loss = loss_fn(all_scores.to(device), all_labels.to(device))

            the_current_loss = the_current_loss + val_loss
            """

            batch_loss = batch_loss + loss
            if counter == batch_size:
                batch_loss = batch_loss / batch_size
                batch_loss.backward(retain_graph=True)
                opt.step()
                opt.zero_grad()
                counter=1
                batch_loss= 0
                h= None
                last_t =t
            else:
                counter += 1
            

        cost = cost / n_timestamps  

        """
        # Early stopping
        
        the_current_loss= the_current_loss / n_timestamps
        if the_current_loss > the_last_loss:
            trigger_times += 1
            #print('Early stopping::trigger times:', trigger_times, end=' ')

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                break
        else:
            #print('Early stopping::trigger times: 0', end=' ')
            trigger_times = 0

        the_last_loss = the_current_loss
        """
        if counter > 1:
            batch_loss= batch_loss / counter
            batch_loss.backward()
            opt.step()
            opt.zero_grad()
        
        #print("Average training & evaluation losses at epoch {} are {:.3f}, {:.3f}".format(epoch, cost, the_current_loss))
        print("Average training loss at epoch {} is {:.3f}".format(epoch, cost))
    
    return model

#### Evaluation of the GNN

In [35]:
def evaluate(model, test_dataset, T, device, GNN_type, max_snapshots=None, eval_dates=None):
    model.eval()

    snapshots_dict= dict((i,j) for i,j in enumerate(test_dataset))
    time_steps= np.sort(list(snapshots_dict.keys()))

    y_true_all =[]
    y_hat_all= []

    results = []
    
    h, c= None, None
    
    #for t in tqdm(time_steps[:-T], desc='Evaluation timesteps...'):
    for t in time_steps[:-T]:

        input_snapshot = snapshots_dict[t].to(device)
        target_snapshot = snapshots_dict[t+T].to(device)

        transform = RandomLinkSplit(is_undirected=False, add_negative_train_samples=False, neg_sampling_ratio=0,
                                        num_val=0.15, num_test=0)

        #train_split, val_split, _ = transform(target_snapshot)

        embeddings, h= model(input_snapshot.x, input_snapshot.edge_index, input_snapshot.edge_attr,h)
        
        neg_edges_to_sample = int(target_snapshot.edge_index.shape[1]*1) #100% of negative edges
        neg_edges = sample_negative_edges(target_snapshot.edge_index, nodes_to_code, neg_edges_to_sample)

        pos_scores= predict_scores(target_snapshot.edge_index, embeddings)
        neg_scores= predict_scores(neg_edges, embeddings)

        all_labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])], dim=0)

        #print(target_snapshot.edge_index.T.shape, neg_edges.T.shape)
        all_scores = torch.cat([pos_scores, neg_scores], dim=0)
        all_labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])], dim=0)
        all_edges = torch.vstack([target_snapshot.edge_index.T.cpu(), neg_edges.T.cpu()])
        #print(all_edges.shape, all_labels.shape)
        edges_and_scores = torch.hstack((all_scores.reshape(-1,1).cpu(),  all_edges.cpu(), all_labels.reshape(-1,1))).cpu().detach().numpy()    
        result_df = pd.DataFrame(edges_and_scores, columns='score origin dest true_label'.split())
        result_df['label_hat']= result_df['score'].apply(lambda v: 1.0 if v>0.5 else 0.0)
        
        if not eval_dates is None:
            result_df['date']= eval_dates[t]
        
        results.append(result_df)
        
        if max_snapshots:
            if t>= max_snapshots:
                break
        
    all_results = pd.concat(results, axis=0)
    return all_results

## Test all combinations

In [ ]:
device_ = "cpu" #torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
max_snapshots_=40
for gnn_type in tqdm(['tour_infras_totals_twt'], desc='GNN type...'):
    for trip_type in tqdm(['high'], desc='Edge type...', leave=False):
        include_twt_=False
        gnn_type_for_loader = gnn_type
        if gnn_type== 'tour_infras_totals_twt':
            include_twt_=True
            gnn_type_for_loader= 'tour_infras_totals'
        loader= TouristMobDynamicDatasetLoader(target_slice=trip_type, edges_type= gnn_type_for_loader, include_twt=include_twt_)
        dataset=loader.get_dataset()
        train_dataset, test_dataset = temporal_signal_split(dataset,  train_ratio=0.8)
        for T_ in tqdm([7,14,21], desc='Time horizon (days)', leave=False):
            model_fitted = train(train_dataset, T_, device_, GNN_type=gnn_type, max_snapshots=max_snapshots_)
            eval_results = evaluate(model_fitted, test_dataset, T_, device_, GNN_type=gnn_type, max_snapshots=max_snapshots_)
            eval_results.to_csv(os.path.join('results', f'eval_results_{gnn_type}_{trip_type}_{T_}_{max_snapshots_}.csv'))

## Use case results generation

In [36]:
tour_flows= pd.read_csv(os.path.join(data_path,'agg_flows_with.csv'), index_col=0, parse_dates=['timestamp'])
tour_flows = tour_flows.set_index('index')

In [37]:
tour_flows.head()

,origen,destino,viajes,timestamp,origin_dest
index,,,,,
0,01001_AM,06060_AM,3.841,2020-04-01,01001_AM_06060_AM
1,01001_AM,09056,10.800,2020-04-01,01001_AM_09056
2,01001_AM,09059,27.477,2020-04-01,01001_AM_09059
3,01001_AM,09194_AM,38.666,2020-04-01,01001_AM_09194_AM
4,01001_AM,09439_AM,10.795,2020-04-01,01001_AM_09439_AM


In [38]:
days= tour_flows['timestamp'].sort_values(ascending=True).unique()
days

array(['2020-04-01T00:00:00.000000000', '2020-04-02T00:00:00.000000000',
       '2020-04-03T00:00:00.000000000', '2020-04-04T00:00:00.000000000',
       '2020-04-05T00:00:00.000000000', '2020-04-06T00:00:00.000000000',
       '2020-04-07T00:00:00.000000000', '2020-04-08T00:00:00.000000000',
       '2020-04-09T00:00:00.000000000', '2020-04-10T00:00:00.000000000',
       '2020-04-11T00:00:00.000000000', '2020-04-12T00:00:00.000000000',
       '2020-04-13T00:00:00.000000000', '2020-04-14T00:00:00.000000000',
       '2020-04-15T00:00:00.000000000', '2020-04-16T00:00:00.000000000',
       '2020-04-17T00:00:00.000000000', '2020-04-18T00:00:00.000000000',
       '2020-04-19T00:00:00.000000000', '2020-04-20T00:00:00.000000000',
       '2020-04-21T00:00:00.000000000', '2020-04-22T00:00:00.000000000',
       '2020-04-23T00:00:00.000000000', '2020-04-24T00:00:00.000000000',
       '2020-04-25T00:00:00.000000000', '2020-04-26T00:00:00.000000000',
       '2020-04-27T00:00:00.000000000', '2020-04-28

In [39]:
len(days)*0.8, len(days)*0.2

(316.0, 79.0)

In [40]:
eval_dates_ = days[317:]

In [41]:
len(eval_dates_)

78

In [ ]:
device_ = "cpu"
max_snapshots_=40
for gnn_type in tqdm(['tour_infras_totals_twt'], desc='GNN type...'):
    for trip_type in tqdm(['low', 'med', 'high'], desc='Edge type...', leave=False):
        include_twt_=False
        gnn_type_for_loader = gnn_type
        if gnn_type== 'tour_infras_totals_twt':
            include_twt_=True
            gnn_type_for_loader= 'tour_infras_totals'
        loader= TouristMobDynamicDatasetLoader(target_slice=trip_type, edges_type= gnn_type_for_loader, include_twt=include_twt_)
        dataset=loader.get_dataset()
        train_dataset, test_dataset = temporal_signal_split(dataset,  train_ratio=0.8)
        
        for T_ in tqdm([14], desc='Time horizon (days)', leave=False):
            model_fitted = train(train_dataset, T_, device_, GNN_type=gnn_type, max_snapshots=max_snapshots_)
            eval_results = evaluate(model_fitted, test_dataset, T_, device_, GNN_type=gnn_type, max_snapshots=max_snapshots_, eval_dates= eval_dates_)
            eval_results.to_csv(os.path.join('results', f'use_case_results_{gnn_type}_{trip_type}_{T_}_{max_snapshots_}.csv'))

GNN type...:   0%|          | 0/1 [00:00<?, ?it/s]

Edge type...:   0%|          | 0/3 [00:00<?, ?it/s]

Time horizon (days):   0%|          | 0/1 [00:00<?, ?it/s]

The number of timesteps for training is 40
Average training loss at epoch 0 is 0.727
Average training loss at epoch 1 is 0.692
Average training loss at epoch 2 is 0.684
Average training loss at epoch 3 is 0.675
Average training loss at epoch 4 is 0.677
Average training loss at epoch 5 is 0.671
Average training loss at epoch 6 is 0.671
Average training loss at epoch 7 is 0.660
Average training loss at epoch 8 is 0.652
Average training loss at epoch 9 is 0.655
Average training loss at epoch 10 is 0.649
Average training loss at epoch 11 is 0.648
Average training loss at epoch 12 is 0.646
Average training loss at epoch 13 is 0.646
Average training loss at epoch 14 is 0.646
Average training loss at epoch 15 is 0.642
Average training loss at epoch 16 is 0.638
Average training loss at epoch 17 is 0.641
Average training loss at epoch 18 is 0.641
Average training loss at epoch 19 is 0.644
Average training loss at epoch 20 is 0.635
Average training loss at epoch 21 is 0.635
Average training loss

Time horizon (days):   0%|          | 0/1 [00:00<?, ?it/s]

The number of timesteps for training is 40
Average training loss at epoch 0 is 0.683
Average training loss at epoch 1 is 0.663
Average training loss at epoch 2 is 0.660
Average training loss at epoch 3 is 0.662
Average training loss at epoch 4 is 0.656
Average training loss at epoch 5 is 0.663
Average training loss at epoch 6 is 0.650
Average training loss at epoch 7 is 0.648
Average training loss at epoch 8 is 0.645
Average training loss at epoch 9 is 0.641
Average training loss at epoch 10 is 0.643
Average training loss at epoch 11 is 0.651
Average training loss at epoch 12 is 0.642
Average training loss at epoch 13 is 0.641
Average training loss at epoch 14 is 0.636
Average training loss at epoch 15 is 0.632
Average training loss at epoch 16 is 0.630
Average training loss at epoch 17 is 0.630
Average training loss at epoch 18 is 0.639
Average training loss at epoch 19 is 0.640
Average training loss at epoch 20 is 0.648
Average training loss at epoch 21 is 0.635
Average training loss

Time horizon (days):   0%|          | 0/1 [00:00<?, ?it/s]

The number of timesteps for training is 40
Average training loss at epoch 0 is 0.693
Average training loss at epoch 1 is 0.691
Average training loss at epoch 2 is 0.679
Average training loss at epoch 3 is 0.652
Average training loss at epoch 4 is 0.643
Average training loss at epoch 5 is 0.638
Average training loss at epoch 6 is 0.643
Average training loss at epoch 7 is 0.646
Average training loss at epoch 8 is 0.647
Average training loss at epoch 9 is 0.644
Average training loss at epoch 10 is 0.645
Average training loss at epoch 11 is 0.647
Average training loss at epoch 12 is 0.648
Average training loss at epoch 13 is 0.641
Average training loss at epoch 14 is 0.640
Average training loss at epoch 15 is 0.647
Average training loss at epoch 16 is 0.640
Average training loss at epoch 17 is 0.636
Average training loss at epoch 18 is 0.648
Average training loss at epoch 19 is 0.632
Average training loss at epoch 20 is 0.629
Average training loss at epoch 21 is 0.636
Average training loss

In [ ]:
print("That's all folks!")